In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("PopularMoviesNicerWithBroadcast")
sc = SparkContext(conf = conf)

In [17]:
movie_counts_raw=sc.textFile("/home/ggomarr/Documents/Education/Udemy_Spark/ml-100k/u.data")
movie_counts_sorted=(movie_counts_raw.map(lambda x: (int(x.split()[1]),1) )
                                     .reduceByKey(lambda x, y: x + y)
                                     .map(lambda x: (x[1], x[0]))
                                     .sortByKey(False))

In [18]:
for movie in movie_counts_sorted.take(10):
    print('{:4d} - {:3d}'.format(movie[0],movie[1]))

 583 -  50
 509 - 258
 508 - 100
 507 - 181
 485 - 294
 481 - 286
 478 - 288
 452 -   1
 431 - 300
 429 - 121


In [20]:
def process_movie_names_file(movie_names_file):
    movie_names_dict={}
    with open(movie_names_file) as f:
        for movie_row in f:
            movie=movie_row.split('|')
            movie_id=int(movie[0])
            movie_title=movie[1]
            movie_names_dict[movie_id]=movie_title
    return movie_names_dict

movie_names_file="/home/ggomarr/Documents/Education/Udemy_Spark/ml-100k/u.item"
movie_names_dict=process_movie_names_file(movie_names_file)

In [21]:
for movie_id in range(1,11):
    print('{:4d} - {}'.format(movie_id,movie_names_dict[movie_id]))

   1 - Toy Story (1995)
   2 - GoldenEye (1995)
   3 - Four Rooms (1995)
   4 - Get Shorty (1995)
   5 - Copycat (1995)
   6 - Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
   7 - Twelve Monkeys (1995)
   8 - Babe (1995)
   9 - Dead Man Walking (1995)
  10 - Richard III (1995)


In [56]:
movie_names_dict_bc=sc.broadcast(movie_names_dict)

In [22]:
movie_counts_with_names_sorted=movie_counts_sorted.map(lambda x: (x[0],(x[1],movie_names_dict_bc.value[x[1]])))

In [24]:
for movie in movie_counts_with_names_sorted.take(10):
    print('{:3d} - {:4d} - {}'.format(movie[0],movie[1][0],movie[1][1]))

583 -   50 - Star Wars (1977)
509 -  258 - Contact (1997)
508 -  100 - Fargo (1996)
507 -  181 - Return of the Jedi (1983)
485 -  294 - Liar Liar (1997)
481 -  286 - English Patient, The (1996)
478 -  288 - Scream (1996)
452 -    1 - Toy Story (1995)
431 -  300 - Air Force One (1997)
429 -  121 - Independence Day (ID4) (1996)
